In [251]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [252]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [253]:
df.shape

(10000, 14)

In [254]:
df.isnull().sum()

,0
RowNumber,0
CustomerId,0
Surname,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0


In [255]:
df.duplicated().sum()

np.int64(0)

In [256]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [257]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [258]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [259]:
X = df.iloc[:,:-1]
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [260]:
y = df.iloc[:,-1]

In [261]:
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [262]:
x_train,x_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=1)

In [263]:
transformer = ColumnTransformer([
    ('encoding', StandardScaler(), ['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'Tenure']),
    ('ohe', OneHotEncoder(sparse_output=False, drop='first'), ['Geography', 'Gender'])
], remainder='passthrough')

In [264]:
x_train = transformer.fit_transform(x_train)

In [265]:
x_train.shape

(8000, 11)

In [266]:
x_test = transformer.transform(x_test)

In [267]:
!pip install keras-tuner

In [268]:
import keras_tuner as kt

In [269]:
def build_model(hp):
  model = Sequential()

  counter =0


  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter==0:
      model.add(Dense(
          hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
          activation= hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid'])
      ))


    else:
      model.add(Dense(
          hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
          activation= hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid'])
      ))


      counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadalta']),
                loss='binary_crossentropy',
                metrics = ['accuracy'] )
  return model

In [270]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='final')

Reloading Tuner from mydir/final/tuner0.json


In [271]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [272]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [273]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9,
 'units0': 64,
 'activation0': 'tanh',
 'dropout0': 0.7,
 'optimizer': 'nadam',
 'units1': 40,
 'activation1': 'tanh',
 'dropout1': 0.2,
 'units2': 8,
 'activation2': 'sigmoid',
 'dropout2': 0.8,
 'units3': 96,
 'activation3': 'tanh',
 'dropout3': 0.5,
 'units4': 40,
 'activation4': 'relu',
 'dropout4': 0.2,
 'units5': 72,
 'activation5': 'tanh',
 'dropout5': 0.9,
 'units6': 96,
 'activation6': 'tanh',
 'dropout6': 0.3,
 'units7': 16,
 'activation7': 'sigmoid',
 'dropout7': 0.1,
 'units8': 8,
 'activation8': 'relu',
 'dropout8': 0.1}

In [274]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 43 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [275]:
model.fit(x_train,y_train,epochs=20,validation_split=0.2)

Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8023 - loss: 0.4633 - val_accuracy: 0.7975 - val_loss: 0.4407
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8111 - loss: 0.4223 - val_accuracy: 0.7975 - val_loss: 0.4385
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8022 - loss: 0.4268 - val_accuracy: 0.7975 - val_loss: 0.4308
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7948 - loss: 0.4306 - val_accuracy: 0.7975 - val_loss: 0.4321
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8012 - loss: 0.4281 - val_accuracy: 0.7975 - val_loss: 0.4201
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7999 - loss: 0.4127 - val_accuracy: 0.7975 - val_loss: 0.3907
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8055 - loss: 0.3882 - val_accuracy: 0.8231 - val_loss: 0.3842
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8419 - loss: 0.3760 - val_accuracy: 0

Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.7969 - loss: 0.4716 - val_accuracy: 0.7975 - val_loss: 0.4458
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.7979 - loss: 0.4325 - val_accuracy: 0.7975 - val_loss: 0.4418
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7954 - loss: 0.4328 - val_accuracy: 0.7975 - val_loss: 0.4437
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8015 - loss: 0.4242 - val_accuracy: 0.7975 - val_loss: 0.4289
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8017 - loss: 0.4148 - val_accuracy: 0.7975 - val_loss: 0.4186
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7980 - loss: 0.4111 - val_accuracy: 0.7975 - val_loss: 0.3916
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8251 - loss: 0.3923 - val_accuracy: 0.8313 - val_loss: 0.3855
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8378 - loss: 0.3904 - val_accuracy